# Lab2: Memory Process Analysis

Identifying Suspicious APT.img file. Notebook run on a [WIN-FOR](https://github.com/digitalsleuth/WIN-FOR) configured machine.

In [29]:
# Setup

from pathlib import Path
import subprocess, hashlib, json, textwrap, os, sys, shlex
IMG = Path(r"C:\Users\vboxuser\Downloads\CYBS-5014\APT\APT.img")
CASE_DIR  = Path.cwd() / "output"
VOL = Path(r"C:\Users\vboxuser\repos\volatility3\vol.py")
if not IMG.is_file():
    raise FileNotFoundError("File does not exist")

# Hashing the APT file

In [25]:
def file_hashes(p: Path, chunksize=2**20):
    h = { 'md5': hashlib.md5(), 'sha256': hashlib.sha256() }
    with p.open('rb') as f:
        while chunk := f.read(chunksize):
            h['md5'].update(chunk); h['sha256'].update(chunk)
    return {k:v.hexdigest() for k,v in h.items()}

hashes = file_hashes(IMG)
(h := (CASE_DIR  / "evidence_hashes.json")).write_text(json.dumps({'image': IMG.name, **hashes}, indent=2))
hashes

{'md5': '14ac99e2e6980d6a962f0ac2a1cb65c8',
 'sha256': '1935771e1d3a3b75cfdf19282332898c7fd7d66b317edd0271ff8f2ed41d69a8'}

# Volatility Wrapper Functions

In [36]:
def vol(plugin, *args):
    cmd = [sys.executable, str(VOL), "-f", str(IMG), plugin, *args]
    out = subprocess.run(cmd, capture_output=True, text=True)
    if out.returncode != 0:
        print(out.stderr)
    return out.stdout

def save(txt, name):
    p = CASE_DIR  / name
    p.write_text(txt)
    return p

# Determine OS/banner, KDBG, and Basic Environment

Running volatility's info/banner plugin to identify:
- OS Family/Version
- Kernel Debugger Block (KDBG)
- Symbol Context

In [37]:
info = vol("windows.info")
save(info, "01_windows.info.txt")
print(info[:1200])

Volatility 3 Framework 2.26.2

Variable	Value

Kernel Base	0x804d7000
DTB	0x319000
Symbols	file:///C:/Users/vboxuser/repos/volatility3/volatility3/symbols/windows/ntkrnlpa.pdb/FD7B256CD2D14BC0B35BA9B89F691346-1.json.xz
Is64Bit	False
IsPAE	True
layer_name	0 WindowsIntelPAE
memory_layer	1 FileLayer
KdDebuggerDataBlock	0x80545b60
NTBuildLab	2600.xpsp_sp3_gdr.080814-1236
CSDVersion	3
KdVersionBlock	0x80545b38
Major/Minor	15.2600
MachineType	332
KeNumberProcessors	1
SystemTime	2009-05-05 19:28:57+00:00
NtSystemRoot	C:\WINDOWS
NtProductType	NtProductWinNt
NtMajorVersion	5
NtMinorVersion	1
PE MajorOperatingSystemVersion	5
PE MinorOperatingSystemVersion	1
PE Machine	332
PE TimeDateStamp	Thu Aug 14 09:33:12 2008



Useful Information:
- Windows NtProductWinNt 5.1 is the kernel version used for Windows XP
- NTBuildLab: xpsp_sp3 refers to Service Pack 3; 080814-1236 refers to the compile time of the build (August 14, 2008)
- Intel x86 x32-bit processor (MachineType 332), single CPU core (KeNumberProcessors)
- Directory Table Base: 0x319000
- Symbols: Structures and functions locations in memory